In [2]:
!pip install torch torchvision torchaudio lightning

In [19]:
a = 1
print(a)

1


In [74]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
import os

SAVE_DIR = r"C:\Users\Users\eye_captures"
os.makedirs(SAVE_DIR, exist_ok=True)



# Mediapipe 초기 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)



# 눈 랜드마크 인덱스 (정확한 좌우 기준)
RIGHT_EYE_IDX = [33, 133, 160, 159, 158, 157, 173, 246]   # 사람의 오른쪽 눈 (화면에서는 왼쪽)
LEFT_EYE_IDX = [362, 263, 387, 386, 385, 384, 398, 466]   # 사람의 왼쪽 눈 (화면에서는 오른쪽)



# 눈 이미지 추출 함수
def extract_eye(image, landmarks, eye_indices, padding=20):
    ih, iw, _ = image.shape
    eye_points = np.array([(int(landmarks[i].x * iw), int(landmarks[i].y * ih)) for i in eye_indices])
    x, y, w, h = cv2.boundingRect(eye_points)

    x = max(x - padding, 0)
    y = max(y - padding, 0)
    w = min(w + padding * 2, iw - x)
    h = min(h + padding * 2, ih - y)

    eye_img = image[y:y+h, x:x+w]
    return eye_img

# 웹캠 시작
cap = cv2.VideoCapture(0)
print("웹캠 실행 중... 's' 키를 누르면 눈을 저장하고, 'q' 키를 누르면 종료됩니다.")

left_eye_img = None
right_eye_img = None

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("카메라 프레임을 가져올 수 없습니다.")
        break

    # RGB 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # 눈 영역 추출
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark
            try:
                left_eye_img = extract_eye(frame, landmarks, LEFT_EYE_IDX)
                right_eye_img = extract_eye(frame, landmarks, RIGHT_EYE_IDX)

                # 눈 이미지 실시간 출력
                if left_eye_img.size > 0:
                    cv2.imshow("Left Eye", left_eye_img)
                if right_eye_img.size > 0:
                    cv2.imshow("Right Eye", right_eye_img)
            except Exception as e:
                print("눈 추출 중 오류 발생:", e)
                continue

    # 원본 영상 표시
    cv2.imshow("Webcam", frame)

    key = cv2.waitKey(1) & 0xFF

    # 눈 저장: 's' 키
    if key == ord('s') and left_eye_img is not None and right_eye_img is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
        left_path = f"left_eye_{timestamp}.png"
        right_path = f"right_eye_{timestamp}.png"

        # ✅ 눈 이미지 90x90으로 리사이즈
        left_resized = cv2.resize(left_eye_img, (90, 90))
        right_resized = cv2.resize(right_eye_img, (90, 90))

        # ✅ 저장
        cv2.imwrite(os.path.join(SAVE_DIR, left_path), left_resized)
        cv2.imwrite(os.path.join(SAVE_DIR, right_path), right_resized)

        print(f"눈 저장 완료: {left_path}, {right_path}")

    # 종료: 'q' 키
    elif key == ord('q'):
        print("종료합니다.")
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()

웹캠 실행 중... 's' 키를 누르면 눈을 저장하고, 'q' 키를 누르면 종료됩니다.
종료합니다.


In [75]:
import os
print("현재 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: C:\Windows\System32


In [26]:
import os
import cv2
import mediapipe as mp
import numpy as np
import random

# Mediapipe 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

# 눈 감김 판별 함수
def is_eye_closed(upper_y, lower_y, threshold=3):
    return abs(upper_y - lower_y) < threshold

# 기준 경로
BASE_PATH = r"C:\Users\Users\open-closed-eyes-dataset"
sets = ['train', 'val', 'test']
labels = ['open', 'closed']
MAX_IMAGES = 1000  # 폴더당 최대 이미지 수

# 결과 저장
results = []

for subset in sets:
    for label in labels:
        folder_path = os.path.join(BASE_PATH, subset, label)
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_files = random.sample(image_files, min(MAX_IMAGES, len(image_files)))  # 랜덤 선택

        print(f"[{subset}/{label}] 이미지 {len(image_files)}장 처리 중...")

        for filename in image_files:
            file_path = os.path.join(folder_path, filename)
            image = cv2.imread(file_path)

            if image is None:
                print(f"  - 이미지 로딩 실패: {filename}")
                continue

            ih, iw, _ = image.shape
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results_mp = face_mesh.process(rgb)

            # 기본 판정: 눈을 감은 상태
            pred_label = "closed"

            if results_mp.multi_face_landmarks:
                try:
                    landmarks = results_mp.multi_face_landmarks[0].landmark
                    # 왼쪽 눈 (386, 374), 오른쪽 눈 (159, 145)
                    l_top = int(landmarks[386].y * ih)
                    l_bottom = int(landmarks[374].y * ih)
                    r_top = int(landmarks[159].y * ih)
                    r_bottom = int(landmarks[145].y * ih)

                    left_closed = is_eye_closed(l_top, l_bottom)
                    right_closed = is_eye_closed(r_top, r_bottom)

                    pred_label = 'closed' if (left_closed and right_closed) else 'open'

                except Exception as e:
                    pred_label = 'closed'  # 에러 발생 시 감은 상태로 간주

            results.append({
                'path': file_path,
                'true_label': label,
                'pred_label': pred_label
            })

# 정확도 계산
total = len(results)
correct = sum(1 for r in results if r['true_label'] == r['pred_label'])
accuracy = (correct / total) * 100 if total > 0 else 0

print("\n📊 예측 결과 요약")
print(f"총 이미지 수     : {total}")
print(f"정확히 예측한 수 : {correct}")
print(f"정확도           : {accuracy:.2f}%")

[train/open] 이미지 1000장 처리 중...
[train/closed] 이미지 1000장 처리 중...
[val/open] 이미지 1000장 처리 중...
[val/closed] 이미지 1000장 처리 중...
[test/open] 이미지 1000장 처리 중...
[test/closed] 이미지 1000장 처리 중...

📊 예측 결과 요약
총 이미지 수     : 6000
정확히 예측한 수 : 3364
정확도           : 56.07%


In [34]:
import os
import cv2
import mediapipe as mp
import numpy as np
import random

# Mediapipe 설정
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

# 눈 감김 판단 함수 (왼/오 자동 감지)
def is_eye_closed_by_landmark(landmarks, image_height, threshold=3):
    try:
        l_top = landmarks[386].y * image_height
        l_bottom = landmarks[374].y * image_height
        if abs(l_top - l_bottom) < threshold:
            return True
    except:
        pass

    try:
        r_top = landmarks[159].y * image_height
        r_bottom = landmarks[145].y * image_height
        if abs(r_top - r_bottom) < threshold:
            return True
    except:
        pass

    return True  # 둘 다 감지 실패 시 감은 것으로 간주

# 데이터셋 경로
base_dir = r"C:\Users\Users\open-closed-eyes-dataset"
sets = ['train', 'val', 'test']
labels = ['open', 'closed']
max_images = 500
landmark_detected = 0

results = []

for subset in sets:
    for label in labels:
        folder = os.path.join(base_dir, subset, label)
        images = [f for f in os.listdir(folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
        images = random.sample(images, min(max_images, len(images)))

        print(f"[{subset}/{label}] {len(images)}장 처리 중...")

        for img_name in images:
            path = os.path.join(folder, img_name)
            img = cv2.imread(path)
            if img is None:
                continue

            ih, iw, _ = img.shape
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = face_mesh.process(rgb)

            if result.multi_face_landmarks:
                landmark_detected += 1

            pred = "closed"  # 기본값은 감은 상태

            if result.multi_face_landmarks:
                landmarks = result.multi_face_landmarks[0].landmark
                closed = is_eye_closed_by_landmark(landmarks, ih)
                pred = "closed" if closed else "open"

            results.append({
                'path': path,
                'true_label': label,
                'pred_label': pred
            })
        print(f"랜드마크 인식 성공 이미지 수: {landmark_detected}/{len(images)}")

[train/open] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 73/500
[train/closed] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 86/500
[val/open] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 172/500
[val/closed] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 183/500
[test/open] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 264/500
[test/closed] 500장 처리 중...
랜드마크 인식 성공 이미지 수: 271/500


In [35]:
total = len(results)
correct = sum(1 for r in results if r['true_label'] == r['pred_label'])
accuracy = (correct / total) * 100 if total > 0 else 0

print(f"\n📊 총 이미지: {total} / 정확히 맞춘 수: {correct} / 정확도: {accuracy:.2f}%")


📊 총 이미지: 3000 / 정확히 맞춘 수: 1500 / 정확도: 50.00%


In [36]:
from collections import Counter

preds = [r['pred_label'] for r in results]
counts = Counter(preds)
print("예측 분포:", counts)

예측 분포: Counter({'closed': 3000})


In [76]:
import os
import cv2
import mediapipe as mp
import numpy as np
import random
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
MAX_PER_CLASS = 2000

# -------------------------
# [1] Mediapipe landmark 추출
# -------------------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

def extract_eye_landmarks(image):
    ih, iw, _ = image.shape
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if not result.multi_face_landmarks:
        return None  # 얼굴 인식 실패

    landmarks = result.multi_face_landmarks[0].landmark
    try:
        left_top = landmarks[386].y * ih
        left_bottom = landmarks[374].y * ih
        right_top = landmarks[159].y * ih
        right_bottom = landmarks[145].y * ih
        return [left_top, left_bottom, right_top, right_bottom]
    except:
        return None  # 좌표 추출 실패

# -------------------------
# [2] 이미지 경로 순회 및 좌표 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=2000):
    X = []
    y = []

    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))

        label = 1 if label_name == 'open' else 0

        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue

            coords = extract_eye_landmarks(img)
            if coords:
                X.append(coords)
                y.append(label)

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

# -------------------------
# [3] PyTorch Dataset 클래스
# -------------------------
class EyeLandmarkDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# -------------------------
# [4] MLP 모델 정의
# -------------------------
class EyeMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.model(x)

# -------------------------
# [5] 데이터 경로 설정 (train/val/test 폴더 각각 MAX_PER_CLASS개씩)
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

X_train, y_train = load_eye_data(train_dir, max_per_class=MAX_PER_CLASS)
X_val, y_val = load_eye_data(val_dir, max_per_class=MAX_PER_CLASS)
X_test, y_test = load_eye_data(test_dir, max_per_class=MAX_PER_CLASS)

print(f"학습 샘플 수: {len(X_train)} / 검증: {len(X_val)} / 테스트: {len(X_test)}")

# -------------------------
# [6] Dataset/DataLoader
# -------------------------
train_loader = DataLoader(EyeLandmarkDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(EyeLandmarkDataset(X_val, y_val), batch_size=32)
test_loader = DataLoader(EyeLandmarkDataset(X_test, y_test), batch_size=32)

# -------------------------
# [7] 모델 학습
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EyeMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("✅ 학습 시작")

for epoch in range(10):
    model.train()
    running_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 검증
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            preds = model(X_batch).argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += len(y_batch)

    val_acc = correct / total * 100
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.4f} | Val Accuracy: {val_acc:.2f}%")

학습 샘플 수: 408 / 검증: 367 / 테스트: 417
✅ 학습 시작
[Epoch 1] Loss: 4.7005 | Val Accuracy: 88.28%
[Epoch 2] Loss: 4.3445 | Val Accuracy: 88.28%
[Epoch 3] Loss: 4.2970 | Val Accuracy: 88.28%
[Epoch 4] Loss: 4.2975 | Val Accuracy: 88.28%
[Epoch 5] Loss: 4.3282 | Val Accuracy: 88.28%
[Epoch 6] Loss: 4.3261 | Val Accuracy: 88.28%
[Epoch 7] Loss: 4.3805 | Val Accuracy: 88.28%
[Epoch 8] Loss: 4.2957 | Val Accuracy: 88.28%
[Epoch 9] Loss: 4.2979 | Val Accuracy: 88.28%
[Epoch 10] Loss: 4.3622 | Val Accuracy: 88.28%


In [80]:
# -------------------------
# [8] 테스트 평가
# -------------------------
print("\n✅ 테스트 데이터셋 평가")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch).argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += len(y_batch)

test_acc = correct / total * 100
print(f"🎯 테스트 정확도: {test_acc:.2f}%")


✅ 테스트 데이터셋 평가
🎯 테스트 정확도: 96.95%


In [70]:
import os
import random
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# -----------------------
# [1] 하이퍼파라미터
# -----------------------
IMG_SIZE = 90
BATCH_SIZE = 32
EPOCHS = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------
# [2] 전처리 정의
# -----------------------
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# -----------------------
# [3] 랜덤 500개 추출 Dataset 클래스
# -----------------------
class CustomEyeDataset(Dataset):
    def __init__(self, root_dir, max_per_class=2000, transform=None):
        self.samples = []
        self.transform = transform
        classes = ['open', 'closed']

        for label, cls in enumerate(classes):
            folder = os.path.join(root_dir, cls)
            files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
            random.shuffle(files)
            files = files[:max_per_class]

            for fname in files:
                self.samples.append((os.path.join(folder, fname), label))

        random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# -----------------------
# [4] 데이터셋 로더
# -----------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir   = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir  = r"C:\Users\Users\open-closed-eyes-dataset\test"

train_loader = DataLoader(CustomEyeDataset(train_dir, transform=transform), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(CustomEyeDataset(val_dir, transform=transform), batch_size=BATCH_SIZE)
test_loader  = DataLoader(CustomEyeDataset(test_dir, transform=transform), batch_size=BATCH_SIZE)

# -----------------------
# [5] CNN 모델 정의
# -----------------------
class EyeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64 * 11 * 11, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.model(x)

model = EyeCNN().to(device)
print("done")

done


In [81]:
# ---------------------------
# 6. 테스트 평가
# ---------------------------
def evaluate(loader):
    model.eval()  # 평가 모드 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total * 100
    
test_acc = evaluate(test_loader)
print(f"\n🎯 테스트 정확도: {test_acc:.2f}%")


🎯 테스트 정확도: 96.95%


In [90]:
import os
import cv2
import mediapipe as mp
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

MAX_PER_CLASS = 1666

# -------------------------
# [1] Mediapipe landmark 추출 (한쪽 눈만)
# -------------------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

def extract_one_eye_landmarks(image):
    ih, iw, _ = image.shape
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if not result.multi_face_landmarks:
        return None  # 얼굴 인식 실패

    landmarks = result.multi_face_landmarks[0].landmark
    try:
        # 오른쪽 눈 위, 아래 좌표만 사용 (예: 159(위), 145(아래))
        top = landmarks[159].y * ih
        bottom = landmarks[145].y * ih
        return [top, bottom]
    except:
        return None  # 좌표 추출 실패

# -------------------------
# [2] 이미지 경로 순회 및 좌표 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=MAX_PER_CLASS):
    X = []
    y = []

    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))

        label = 1 if label_name == 'open' else 0

        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue

            coords = extract_one_eye_landmarks(img)
            if coords:
                X.append(coords)
                y.append(label)

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

# -------------------------
# [3] PyTorch Dataset 클래스
# -------------------------
class EyeLandmarkDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# -------------------------
# [4] MLP 모델 정의 (입력 노드 2개로 변경)
# -------------------------
class EyeMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 32),   # 입력 노드 2개 -> 은닉층 32개 노드
            nn.ReLU(),          # 비선형 활성화 함수 ReLU
            nn.Linear(32, 16),  # 은닉층 32개 -> 은닉층 16개
            nn.ReLU(),          # ReLU 활성화
            nn.Linear(16, 2)    # 은닉층 16개 -> 출력층 2개 (open/closed 분류)
        )

    def forward(self, x):
        return self.model(x)

# -------------------------
# [5] 데이터 경로 설정 및 데이터 로딩
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

X_train, y_train = load_eye_data(train_dir, max_per_class=MAX_PER_CLASS)
X_val, y_val = load_eye_data(val_dir, max_per_class=MAX_PER_CLASS)
X_test, y_test = load_eye_data(test_dir, max_per_class=MAX_PER_CLASS)

print(f"학습 샘플 수: {len(X_train)} / 검증: {len(X_val)} / 테스트: {len(X_test)}")

# -------------------------
# [6] Dataset/DataLoader 생성
# -------------------------
train_loader = DataLoader(EyeLandmarkDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(EyeLandmarkDataset(X_val, y_val), batch_size=32)
test_loader = DataLoader(EyeLandmarkDataset(X_test, y_test), batch_size=32)

# -------------------------
# [7] 모델 학습
# -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EyeMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("✅ 학습 시작")

for epoch in range(10):
    model.train()
    running_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            preds = model(X_batch).argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += len(y_batch)

    val_acc = correct / total * 100
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.4f} | Val Accuracy: {val_acc:.2f}%")

학습 샘플 수: 339 / 검증: 325 / 테스트: 288
✅ 학습 시작
[Epoch 1] Loss: 4.3233 | Val Accuracy: 92.92%
[Epoch 2] Loss: 4.2116 | Val Accuracy: 92.92%
[Epoch 3] Loss: 4.2671 | Val Accuracy: 92.92%
[Epoch 4] Loss: 4.1870 | Val Accuracy: 92.92%
[Epoch 5] Loss: 4.3676 | Val Accuracy: 92.92%
[Epoch 6] Loss: 4.1704 | Val Accuracy: 92.92%
[Epoch 7] Loss: 4.6892 | Val Accuracy: 92.92%
[Epoch 8] Loss: 4.2520 | Val Accuracy: 92.92%
[Epoch 9] Loss: 4.2774 | Val Accuracy: 92.92%
[Epoch 10] Loss: 4.2715 | Val Accuracy: 92.92%


In [98]:
# -------------------------
# [8] 테스트 평가
# -------------------------
print("\n✅ 테스트 데이터셋 평가")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch).argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += len(y_batch)

test_acc = correct / total * 100
print(f"🎯 테스트 정확도: {test_acc:.2f}%")


✅ 테스트 데이터셋 평가
🎯 테스트 정확도: 89.58%


In [89]:
import os
# test - closed 1666개
# test - open 5325개
# train - closed 33322개
# train - open 106482개
# val - closed 6665 개
# val - open 21296 개
folder_path = r"C:\Users\Users\open-closed-eyes-dataset\val\closed"

# 이미지 확장자 리스트
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

files = [f for f in os.listdir(folder_path) if f.lower().endswith(image_extensions)]
print(f"이미지 파일 개수: {len(files)}")

이미지 파일 개수: 6665


In [99]:
import os
import cv2
import numpy as np
import random

MAX_PER_CLASS = 1666

# -------------------------
# [1] 눈 영역에서 위, 아래 y 좌표 추출 (얼굴 인식 제거)
# -------------------------
def extract_one_eye_landmarks_direct(image):
    ih, iw, _ = image.shape

    # 눈 영역만 있다고 가정, 이미지 높이를 3등분하여 위, 아래 좌표 추출 예시
    top = ih * 1/3
    bottom = ih * 2/3
    return [top, bottom]

# -------------------------
# [2] 이미지 경로 순회 및 좌표 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=MAX_PER_CLASS):
    data = []

    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))

        label = 1 if label_name == 'open' else 0

        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue

            coords = extract_one_eye_landmarks_direct(img)
            if coords:
                top, bottom = coords
                distance = bottom - top
                data.append((distance, label))

    return data

# -------------------------
# [3] 간단한 임계값 기반 분류
# -------------------------
def simple_threshold_classifier(data, threshold):
    correct = 0
    total = 0

    for distance, label in data:
        pred = 0 if distance < threshold else 1  # 거리 작으면 감김(closed=0), 크면 뜸(open=1)
        if pred == label:
            correct += 1
        total += 1

    accuracy = correct / total * 100
    return accuracy

# -------------------------
# [4] 데이터 경로 설정 및 데이터 로딩
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

train_data = load_eye_data(train_dir)
val_data = load_eye_data(val_dir)
test_data = load_eye_data(test_dir)

print(f"학습 샘플 수: {len(train_data)} / 검증: {len(val_data)} / 테스트: {len(test_data)}")

# -------------------------
# [5] 임계값 설정 (예: 경험적으로 정하거나 검증 데이터에서 탐색)
# -------------------------
threshold = 50  # 픽셀 단위 임의 값, 적절히 조정 필요

# -------------------------
# [6] 정확도 평가
# -------------------------
train_acc = simple_threshold_classifier(train_data, threshold)
val_acc = simple_threshold_classifier(val_data, threshold)
test_acc = simple_threshold_classifier(test_data, threshold)

print(f"Train Accuracy: {train_acc:.2f}%")
print(f"Validation Accuracy: {val_acc:.2f}%")
print(f"Test Accuracy: {test_acc:.2f}%")

학습 샘플 수: 3332 / 검증: 3332 / 테스트: 3332
Train Accuracy: 50.00%
Validation Accuracy: 50.00%
Test Accuracy: 50.00%


In [100]:
import os
import cv2
import numpy as np
import random

MAX_PER_CLASS = 1666

# -------------------------
# [1] 눈 이미지에서 눈꺼풀 위·아래 y 좌표 직접 추출
# -------------------------
def extract_eye_top_bottom(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 눈동자는 보통 어두워서 역이진화 (어두운 영역을 흰색으로)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # 수평 픽셀 합 (y축 방향)
    horizontal_sum = np.sum(thresh, axis=1)

    ys = np.where(horizontal_sum > 0)[0]

    if len(ys) == 0:
        return None

    top = ys[0]
    bottom = ys[-1]

    return top, bottom

# -------------------------
# [2] 이미지 폴더 순회하며 데이터(눈꺼풀 거리, 라벨) 수집
# -------------------------
def load_eye_data(base_dir, max_per_class=MAX_PER_CLASS):
    data = []

    for label_name in ['open', 'closed']:
        folder = os.path.join(base_dir, label_name)
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png'))]
        files = random.sample(files, min(max_per_class, len(files)))

        label = 1 if label_name == 'open' else 0

        for file in files:
            path = os.path.join(folder, file)
            img = cv2.imread(path)
            if img is None:
                continue

            coords = extract_eye_top_bottom(img)
            if coords:
                top, bottom = coords
                distance = bottom - top
                data.append((distance, label))

    return data

# -------------------------
# [3] 임계값 기반 눈 감김 분류기
# -------------------------
def simple_threshold_classifier(data, threshold):
    correct = 0
    total = 0

    for distance, label in data:
        pred = 0 if distance < threshold else 1
        if pred == label:
            correct += 1
        total += 1

    accuracy = correct / total * 100 if total > 0 else 0
    return accuracy

# -------------------------
# [4] 데이터 경로 설정 및 데이터 로딩
# -------------------------
train_dir = r"C:\Users\Users\open-closed-eyes-dataset\train"
val_dir = r"C:\Users\Users\open-closed-eyes-dataset\val"
test_dir = r"C:\Users\Users\open-closed-eyes-dataset\test"

train_data = load_eye_data(train_dir)
val_data = load_eye_data(val_dir)
test_data = load_eye_data(test_dir)

print(f"학습 샘플 수: {len(train_data)} / 검증: {len(val_data)} / 테스트: {len(test_data)}")

# -------------------------
# [5] 경험적으로 정한 임계값 (픽셀 단위, 눈 이미지 크기에 따라 조정 필요)
# -------------------------
threshold = 15

# -------------------------
# [6] 정확도 평가
# -------------------------
train_acc = simple_threshold_classifier(train_data, threshold)
val_acc = simple_threshold_classifier(val_data, threshold)
test_acc = simple_threshold_classifier(test_data, threshold)

print(f"Train Accuracy: {train_acc:.2f}%")
print(f"Validation Accuracy: {val_acc:.2f}%")
print(f"Test Accuracy: {test_acc:.2f}%")

학습 샘플 수: 2766 / 검증: 2714 / 테스트: 2719
Train Accuracy: 52.31%
Validation Accuracy: 53.35%
Test Accuracy: 53.51%
